## MATRIX INVERSION AS A QUBO PROBLEM

Ref  
[1.Floating-Point Calculations on a Quantum Annealer:
Division and Matrix Inversion](https://arxiv.org/pdf/1901.06526.pdf)

In this section we present an algorithm for solving a system of linear equations on a
quantum annealer. To precisely define the mathematical problem, let $M$ be a nonsingular $N × N$ real matrix, and let $Y$ be a real $N$ dimensional vector; we then wish to solve the linear equation
$$ M · x = Y$$
The linearity of the system means that there is a unique solution,
$$x = M^{−1} · Y$$

Constructing a quadratic matrix 
$$H(x) = (M x − Y)^2 = (M x − Y)^T · (M x − Y)$$
$$H(x) = x^T M^T Mx - x^T M^T Y - Y^T M x +Y^T Y$$
$$H(x) = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2\sum_{ij=1}^N Y_j M_{ji} x^i + ||Y^2||$$
To obtain a floating point representation of each component
of $x = (x_1, · · · , x_N)^T$ by expanding in powers of 2 multiplied by Boolean-valued variables
$$\chi^i = \sum_{r=0}^{R-1} 2^{-r}q_r^i $$
$$x^i = 2\chi^i-1$$

And to obtain integer representation, the real value $x_i$

$$ x_i = \sum_{l=-m}^m 2^{-l} q_{i,l}$$


As before, the domains are given by $\chi^i\in [0, 2)$ and $x^i \in [−1, 3)$, and upon expressing $x$ as a function the $q_r^i$ we can recast $H(x)$ in the form  
$$H(q) = \sum_{i=1}^{N}\sum_{r=0}^{R-1} a_r^i q_r^j + \sum_{i=1}^N \sum_{i\neq j=1}^N \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_{r}^i q_s^i$$

where $N$ is dimention of coefficient matrix and $R$ is resolution of qbit
$$a_r^i = \sum_{k=0}^{N-1}\sum_{i=0}^{N-1}\sum_{r=0}^{R-1}2^{1-r}(a_{ki}^2-a_{ki}b_k) - \sum_{k=0}^{N-1}\sum_{i=0}^{N-1}\sum_{r=0}^{R-1}\sum_{j>i}^{N-1}2^{2-r}a_{ki}a_{kj}$$

$$b_{rs}^{ij} = \sum_{k=0}^{N-1}\sum_{i=0}^{N-1}\sum_{j>i}^{N-1}\sum_{r=0}^{R-1}\sum_{s=0}^{R-1}2^{2-r-s}a_{ki}a_{kj}$$

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import random, math
import copy

Dimension = 1
qubits = 4
N = Dimension
R = qubits
M = np.array([[0.5, 1.5], [1.5, 0.5]])
Y = np.array([0,1])



## Decomposition of single float variable

In [4]:
a = 1
b = -1

r_qbit = 4
a_l = {}
b_lm = {}

qbit_list = []
for val in range(r_qbit):
    qbit_list.append('q'+str(val+1)) 
print(qbit_list)

for r in range(r_qbit):
    a_l[(qbit_list[r],qbit_list[r])] = 2*pow(a,2)*pow(2,-2*r)-4*pow(a,2)*pow(2,-r)+4*a*b*pow(2,-r)

for r in range(r_qbit-1):
    for s in range(r+1,r_qbit):
            b_lm[(qbit_list[r],qbit_list[s])] = 2*pow(a,2)*pow(2,-r-s)

print(a_l)
print(b_lm)

['q1', 'q2', 'q3', 'q4']
{('q1', 'q1'): -6, ('q2', 'q2'): -3.5, ('q3', 'q3'): -1.875, ('q4', 'q4'): -0.96875}
{('q1', 'q2'): 1.0, ('q1', 'q3'): 0.5, ('q1', 'q4'): 0.25, ('q2', 'q3'): 0.25, ('q2', 'q4'): 0.125, ('q3', 'q4'): 0.0625}


In [116]:
l_max = N*R
print(l_max)
a_l = {}
QM = np.zeros((qubits*Dimension, qubits*Dimension))
# a_l[0][1] = 2
# print(a_l)
b_lm = {}

qbit_list = []
for val in range(N*R):
    qbit_list.append('q'+str(val+1)) 
print(qbit_list)

for r in range(R):
    for s in range(R):
        for k in range(N):
            for i in range(N-1):
                for j in range(i+1,N):
                    # if i!=j:
                    pos1=i*R+r
                    pos2=j*R+s
                    QM[pos1][pos2]= QM[pos1][pos2]+8*pow(2,-(r+s))*M[k][i]+M[k][j]


for i in range(N-1):
    for r in range(R):
        for s in range(R):
            b_lm[(qbit_list[i*R+r],qbit_list[(i+1)*R+s])]=QM[r][R+s]


for r in range(R):
    for i in range(N):
        pos = (qbit_list[i*R+r],qbit_list[i*R+r])
        if pos not in a_l:
            a_l[pos] = 0
        for k in range(N):
            for j in range(N):
                a_l[pos] = a_l[pos]+M[k][j]
            a_l[pos] = a_l[pos]+Y[k]
            a_l[pos] =pow(2,-r)*M[k][i]- a_l[pos]
            a_l[pos] = a_l[pos]*M[k][i]
        a_l[pos] = a_l[pos]*4*pow(2,-r)

for r in range(R):
    for s in range(r+1,R):
        for k in range(N):
            b_lm[(qbit_list[r],qbit_list[s])]=4*pow(2,-(r+s))*pow(M[k][k],2)

    
print(a_l)
print(b_lm)

print(QM)



4
['q1', 'q2', 'q3', 'q4']
{('q1', 'q1'): -4, ('q2', 'q2'): -3.0, ('q3', 'q3'): -1.75, ('q4', 'q4'): -0.9375}
{('q1', 'q2'): 2.0, ('q1', 'q3'): 1.0, ('q1', 'q4'): 0.5, ('q2', 'q3'): 0.5, ('q2', 'q4'): 0.25, ('q3', 'q4'): 0.125}
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Apply QUBO

In [35]:
qbit_list = []
for val in range(N*R):
    qbit_list.append('q'+str(val+1)) 
print(qbit_list)
Q2 = {}
for i in range(len(qbit_list)):
    for j in range(len(qbit_list)):
        if QM[i][j] != 0:
            Q2[(qbit_list[i],qbit_list[j])] = QM[i][j]
            
print(Q2)
            

['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8']
{('q1', 'q1'): -4.0, ('q1', 'q2'): 2.5, ('q1', 'q3'): 1.25, ('q1', 'q4'): 0.625, ('q1', 'q6'): 1.5, ('q1', 'q7'): 0.75, ('q1', 'q8'): 0.375, ('q2', 'q2'): -2.0, ('q2', 'q3'): 0.625, ('q2', 'q4'): 0.3125, ('q2', 'q5'): 0.75, ('q2', 'q7'): 0.375, ('q2', 'q8'): 0.1875, ('q3', 'q3'): -1.0, ('q3', 'q4'): 0.15625, ('q3', 'q5'): 0.375, ('q3', 'q6'): 0.1875, ('q3', 'q8'): 0.09375, ('q4', 'q4'): -0.5, ('q4', 'q5'): 0.1875, ('q4', 'q6'): 0.09375, ('q4', 'q7'): 0.046875, ('q5', 'q5'): 4.0, ('q5', 'q6'): 2.5, ('q5', 'q7'): 1.25, ('q5', 'q8'): 0.625, ('q6', 'q6'): 2.0, ('q6', 'q7'): 0.625, ('q6', 'q8'): 0.3125, ('q7', 'q7'): 1.0, ('q7', 'q8'): 0.15625, ('q8', 'q8'): 0.5}


In [6]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','q8'):20.0}

Q = dict(linear)
Q.update(quadratic)
Q3 = dict(a_l)
Q3.update(b_lm)

# print(Q)

sampleset = sampler_auto.sample_qubo(Q3, num_reads=1000, chain_strength=20)
print(sampleset)

   q1 q2 q3 q4    energy num_oc. chain_.
0   1  1  1  1 -10.15625     141     0.0
1   1  1  1  0    -9.625     267     0.0
2   1  1  0  1  -9.09375      95     0.0
3   1  1  0  0      -8.5     173     0.0
4   1  0  1  1  -8.03125      39     0.0
5   1  0  1  0    -7.375      68     0.0
6   1  0  0  1  -6.71875      19     0.0
7   1  0  0  0      -6.0      28     0.0
8   0  1  1  1  -5.90625      38     0.0
16  0  1  1  1  -5.90625       1    0.25
9   0  1  1  0    -5.125      74     0.0
10  0  1  0  1  -4.34375      19     0.0
11  0  1  0  0      -3.5      20     0.0
12  0  0  1  1  -2.78125       3     0.0
13  0  0  1  0    -1.875      10     0.0
14  0  0  0  1  -0.96875       2     0.0
15  0  0  0  0       0.0       3     0.0
['BINARY', 17 rows, 1000 samples, 4 variables]


### Convert Qbit to Decimal 

In [8]:
samples = sampleset.samples()
# print(samples[0])
Dimension = 1 
qbit_per_var_count = int(len(qbit_list)/Dimension)

qbit_per_var = []
for i in range(Dimension):
    qbit_per_var.append(samples[i,qbit_list[qbit_per_var_count*i:qbit_per_var_count*(i+1)]])

print(qbit_per_var)
res = []

for i in range(1):
    res.append(0)
    for j in range(4):
        res[i] += (pow(2,-j)*qbit_per_var[i][j])
    res[i] = 2*res[i]-1

print("Output: ", res)
print("input A: ", a," b: ", b)
    


[array([1, 1, 1, 1], dtype=int8)]
Output:  [2.75]
input A:  1  b:  -1


In [66]:
import dimod

#print(Q)
J = dimod.qubo_to_ising(Q2)
print(J)

({'q1': -0.25, 'q2': 0.1875, 'q3': 0.171875, 'q4': 0.10546875, 'q6': 2.3046875, 'q7': 1.30078125, 'q8': 0.6875, 'q5': 3.421875}, {('q1', 'q2'): 0.625, ('q1', 'q3'): 0.3125, ('q1', 'q4'): 0.15625, ('q1', 'q6'): 0.375, ('q1', 'q7'): 0.1875, ('q1', 'q8'): 0.09375, ('q2', 'q3'): 0.15625, ('q2', 'q4'): 0.078125, ('q2', 'q5'): 0.1875, ('q2', 'q7'): 0.09375, ('q2', 'q8'): 0.046875, ('q3', 'q4'): 0.0390625, ('q3', 'q5'): 0.09375, ('q3', 'q6'): 0.046875, ('q3', 'q8'): 0.0234375, ('q4', 'q5'): 0.046875, ('q4', 'q6'): 0.0234375, ('q4', 'q7'): 0.01171875, ('q5', 'q6'): 0.625, ('q5', 'q7'): 0.3125, ('q5', 'q8'): 0.15625, ('q6', 'q7'): 0.15625, ('q6', 'q8'): 0.078125, ('q7', 'q8'): 0.0390625}, 3.96484375)


In [42]:
sampleset = sampler_auto.sample_ising(J[0], J[1], num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8    energy num_oc. chain_.
0  -1 +1 +1 +1 -1 +1 +1 +1 -5.992188     338     0.0
1  -1 +1 +1 +1 -1 +1 +1 -1 -5.492188      57     0.0
2  -1 +1 +1 +1 -1 +1 -1 +1 -4.992188      32     0.0
3  -1 +1 +1 -1 -1 +1 +1 +1 -4.992188      28     0.0
4  +1 -1 +1 +1 -1 +1 +1 +1 -4.992188      83     0.0
5  +1 +1 +1 +1 -1 -1 +1 +1 -4.992188      41     0.0
6  +1 -1 +1 +1 -1 +1 +1 -1 -4.867188      55     0.0
7  +1 +1 +1 -1 -1 -1 +1 +1 -4.867188      34     0.0
8  +1 +1 -1 +1 -1 -1 +1 +1 -4.742188      13     0.0
9  +1 -1 +1 +1 -1 +1 -1 +1 -4.742188      27     0.0
10 +1 +1 -1 +1 -1 +1 -1 +1 -4.617188      16     0.0
11 +1 -1 +1 -1 -1 +1 +1 +1 -4.617188      32     0.0
12 +1 +1 +1 +1 -1 -1 +1 -1 -4.617188      17     0.0
13 +1 +1 +1 -1 -1 +1 -1 +1 -4.554688      15     0.0
14 +1 +1 -1 +1 -1 +1 +1 -1 -4.554688      16     0.0
15 +1 -1 +1 +1 -1 -1 +1 +1 -4.492188      15     0.0
16 +1 +1 -1 +1 -1 +1 +1 +1 -4.492188      10     0.0
17 +1 +1 +1 +1 -1 +1 -1 +1 -4.492188      11  